# **Task 1**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from collections import deque
import heapq
import matplotlib.pyplot as plt

class Node:
    def __init__(self, data, level, fval, action=None, parent=None, step_count=0):
        self.data = data
        self.level = level
        self.fval = fval
        self.action = action
        self.parent = parent
        self.step_count = step_count

    def __lt__(self, other):
        return self.fval < other.fval

    def generate_child(self):
        x, y = self.find(self.data, '_')
        val_list = [[x, y - 1, 'Right'], [x, y + 1, 'Left'], [x - 1, y, 'Down'], [x + 1, y, 'Up']]
        children = []
        for i in val_list:
            child = self.shuffle(self.data, x, y, i[0], i[1])
            if child is not None:
                child_node = Node(child, self.level + 1, 0, action=i[2], parent=self, step_count=self.step_count + 1)
                children.append(child_node)
        return children

    def shuffle(self, puz, x1, y1, x2, y2):
        if 0 <= x2 < len(self.data) and 0 <= y2 < len(self.data[0]):
            temp_puz = [row[:] for row in puz]
            temp = temp_puz[x2][y2]
            temp_puz[x2][y2] = temp_puz[x1][y1]
            temp_puz[x1][y1] = temp
            return temp_puz
        else:
            return None

    def find(self, puz, x):
        for i in range(len(puz)):
            for j in range(len(puz[i])):
                if puz[i][j] == x:
                    return i, j

# Pseudocode for class Node:
# class Node:
#     function __init__(data, level, fval, action=None, parent=None, step_count=0):
#         self.data ← data
#         self.level ← level
#         self.fval ← fval
#         self.action ← action
#         self.parent ← parent
#         self.step_count ← step_count

#     function __lt__(other):
#         return self.fval < other.fval

#     function generate_child():
#         x, y ← FIND(self.data, '_')
#         val_list ← [[x, y - 1, 'Right'], [x, y + 1, 'Left'], [x - 1, y, 'Down'], [x + 1, y, 'Up']]
#         children ← EMPTY LIST
#         for each i in val_list:
#             child ← SHUFFLE(self.data, x, y, i[0], i[1])
#             if child is not None:
#                 child_node ← Node(child, self.level + 1, 0, action=i[2], parent=self, step_count=self.step_count + 1)
#                 APPEND child_node to children
#         return children

#     function shuffle(puz, x1, y1, x2, y2):
#         if 0 <= x2 < LENGTH(self.data) and 0 <= y2 < LENGTH(self.data[0]):
#             temp_puz ← COPY puz
#             temp ← temp_puz[x2][y2]
#             temp_puz[x2][y2] ← temp_puz[x1][y1]
#             temp_puz[x1][y1] ← temp
#             return temp_puz
#         else:
#             return None

#     function find(puz, x):
#         for i in range(LENGTH(puz)):
#             for j in range(LENGTH(puz[i])):
#                 if puz[i][j] == x:
#                     return i, j

class Puzzle:
    def __init__(self, size):
        self.n = size
        self.open = []  # Used for A*
        self.closed = []  # Used for A*
        self.path_cost = 0  # Initialize path cost for A*
        self.total_cost = 0  # Initialize total cost for A*
        self.path_costs = []  # Store path costs for multiple runs

    def accept(self):
        puz = []
        for i in range(self.n):
            temp = input().split(" ")
            puz.append(temp)
        return puz

    def f(self, start, goal):
        return self.h(start.data, goal) + start.level

    def h(self, start, goal):
        temp = 0
        for i in range(self.n):
            for j in range(self.n):
                if start[i][j] != goal[i][j] and start[i][j] != '_':
                    temp += 1
        return temp

    def process_bfs(self, start, goal):
        start = Node(start, 0, 0)
        open_bfs = deque([start])  # Used for BFS
        closed_bfs = set()  # Used to store visited nodes
        path_cost_bfs = 0  # Initialize path cost for BFS

        while open_bfs:
            cur = open_bfs.popleft()
            print("==================================================\n")
            for i in cur.data:
                print(*i)
            if self.h(cur.data, goal) == 0:
                print("Actions taken:")
                actions = self.get_actions(cur)
                print(actions)
                print("Step: ", cur.step_count)
                self.path_costs.append(path_cost_bfs)
                self.total_cost = path_cost_bfs + cur.step_count
                print("Path cost:", self.path_cost)
                print("Total cost:", self.total_cost)
                break
            for i in cur.generate_child():
                if i not in closed_bfs:  # Check if node is visited using set
                    open_bfs.append(i)
                    closed_bfs.add(i)  # Add node to visited set
            path_cost_bfs += 1

    def get_actions(self, node):
        actions = []
        while node.action:
            actions.append(node.action)
            node = node.parent
        actions.reverse()
        return actions

    def process_astar(self, start, goal):
        start = Node(start, 0, 0)
        start.fval = self.f(start, goal)
        heapq.heappush(self.open, (start.fval, start))
        path_cost_astar = 0  # Initialize path cost for A*

        while self.open:
            cur = heapq.heappop(self.open)[1]
            print("==================================================\n")
            for i in cur.data:
                print(*i)
            if self.h(cur.data, goal) == 0:
                print("Actions :")
                actions = self.get_actions(cur)
                print(actions)
                print("Step: ", cur.step_count)
                self.path_costs.append(path_cost_astar)
                self.total_cost = path_cost_astar + cur.step_count
                print("Path cost:", self.path_cost)
                print("Total cost:", self.total_cost)
                break
            for i in cur.generate_child():
                if i not in self.closed:
                    i.fval = self.f(i, goal)
                    heapq.heappush(self.open, (i.fval, i))
            self.closed.append(cur)
            path_cost_astar += 1

# Pseudocode for class Puzzle:
# class Puzzle:
#     function __init__(size):
#         self.n ← size
#         self.open ← EMPTY LIST
#         self.closed ← EMPTY LIST
#         self.path_cost ← 0
#         self.total_cost ← 0
#         self.path_costs ← EMPTY LIST

#     function accept():
#         puz ← EMPTY LIST
#         for i in range(self.n):
#             temp ← SPLIT USER INPUT BY SPACE
#             APPEND temp to puz
#         return puz

#     function f(start, goal):
#         return h(start.data, goal) + start.level

#     function h(start, goal):
#         temp ← 0
#         for i in range(self.n):
#             for j in range(self.n):
#                 if start[i][j] != goal[i][j] and start[i][j] != '_':
#                     temp ← temp + 1
#         return temp

#     function process_bfs(start, goal):
#         start ← Node(start, 0, 0)
#         open_bfs ← QUEUE WITH start
#         closed_bfs ← EMPTY SET
#         path_cost_bfs ← 0
#         while open_bfs is not EMPTY:
#             cur ← DEQUEUE open_bfs
#             PRINT cur.data
#             if h(cur.data, goal) == 0:
#                 PRINT "Actions taken:"
#                 actions ← get_actions(cur)
#                 PRINT actions
#                 PRINT "Step: ", cur.step_count
#                 APPEND path_cost_bfs to self.path_costs
#                 self.total_cost ← path_cost_bfs + cur.step_count
#                 PRINT "Path cost:", self.path_cost
#                 PRINT "Total cost:", self.total_cost
#                 break
#             for each i in cur.generate_child():
#                 if i not in closed_bfs:
#                     ENQUEUE i to open_bfs
#                     ADD i to closed_bfs
#             path_cost_bfs ← path_cost_bfs + 1

#     function get_actions(node):
#         actions ← EMPTY LIST
#         while node.action is not None:
#             APPEND node.action to actions
#             node ← node.parent
#         REVERSE actions
#         return actions

#     function process_astar(start, goal):
#         start ← Node(start, 0, 0)
#         start.fval ← f(start, goal)
#         PUSH (start.fval, start) to self.open
#         path_cost_astar ← 0
#         while self.open is not EMPTY:
#             cur ← POP MINIMUM ELEMENT from self.open
#             PRINT cur.data
#             if h(cur.data, goal) == 0:
#                 PRINT "Actions :"
#                 actions ← get_actions(cur)
#                 PRINT actions
#                 PRINT "Step: ", cur.step_count
#                 APPEND path_cost_astar to self.path_costs
#                 self.total_cost ← path_cost_astar + cur.step_count
#                 PRINT "Path cost:", self.path_cost
#                 PRINT "Total cost:", self.total_cost
#                 break
#             for each i in cur.generate_child():
#                 if i not in self.closed:
#                     i.fval ← f(i, goal)
#                     PUSH (i.fval, i) to self.open
#             APPEND cur to self.closed
#             path_cost_astar ← path_cost_astar + 1

if __name__ == "__main__":
    size = 3
    puz = Puzzle(size)

    print("Enter the start state matrix: ")
    start_state = puz.accept()
    print("Enter the goal state matrix: ")
    goal_state = puz.accept()

    while True:
        choice = input("Choose algorithm: (1) BFS or (2) A*: ")
        if choice == '1':
            print("\nSolving with BFS:")
            puz.process_bfs(start_state, goal_state)
            break
        elif choice == '2':
            print("\nSolving with A*:")
            puz.process_astar(start_state, goal_state)
            break
        else:
            print("Invalid choice. Please choose 1 for BFS or 2 for A*.")

    # Calculate and display average path costsS
    average_path_cost_bfs = sum(puz.path_costs) / len(puz.path_costs)
    average_path_cost_astar = sum(puz.path_costs) / len(puz.path_costs)  # Calculate average path cost for A* similarly

    print("Average Path Cost (BFS):", average_path_cost_bfs)
    print("Average Path Cost (A*):", average_path_cost_astar)

    # Draw a bar chart to illustrate average path costs
    algorithms = ['BFS', 'A*']
    average_path_costs = [average_path_cost_bfs, average_path_cost_astar]  # Assuming average_path_cost_astar is calculated similarly

    plt.bar(algorithms, average_path_costs, color=['blue', 'green'])
    plt.xlabel('Algorithms')
    plt.ylabel('Average Path Cost')
    plt.title('Average Path Costs for BFS and A*')
    plt.show()

# Pseudocode for function main():
# if __name__ == "__main__":
#     size ← 3
#     puz ← Puzzle(size)
#     PRINT "Enter the start state matrix: "
#     start_state ← puz.accept()
#     PRINT "Enter the goal state matrix: "
#     goal_state ← puz.accept()
#     while True:
#         choice ← USER INPUT
#         if choice == '1':
#             PRINT "\nSolving with BFS:"
#             puz.process_bfs(start_state, goal_state)
#             break
#         elif choice == '2':
#             PRINT "\nSolving with A*:"
#             puz.process_astar(start_state, goal_state)
#             break
#         else:
#             PRINT "Invalid choice. Please choose 1 for BFS or 2 for A*."
#     average_path_cost_bfs ← SUM(puz.path_costs) / LENGTH(puz.path_costs)
#     average_path_cost_astar ← SUM(puz.path_costs) / LENGTH(puz.path_costs)
#     PRINT "Average Path Cost (BFS):", average_path_cost_bfs
#     PRINT "Average Path Cost (A*):", average_path_cost_astar
#     algorithms ← ['BFS', 'A*']
#     average_path_costs ← [average_path_cost_bfs, average_path_cost_astar]
#     DRAW BAR CHART with algorithms on x-axis and average_path_costs on y-axis
#     SET x-label to 'Algorithms'
#     SET y-label to 'Average Path Cost'
#     SET title to 'Average Path Costs for BFS and A*'
#     SHOW PLOT

Enter the start state matrix: 
132
31
12
Enter the goal state matrix: 
213
13
1313
Choose algorithm: (1) BFS or (2) A*: 1

Solving with BFS:

132
31
12


IndexError: list index out of range

In [ ]:

---------------------------------------------------------------------------

function ucs(graph, start, goal):
    frontier = PriorityQueue()
    frontier.push(start, 0)
    explored = set()

    while not frontier.empty():
        current_node = frontier.pop()
        print(current_node.data)

        if current_node.data == goal:
            actions = get_actions_list(current_node)
            print("Actions:", actions)
            print("Step:", current_node.step_count)
            return

        explored.add(current_node.data)

        for neighbor in current_node.generate_child():
            if neighbor not in explored:
                neighbor_cost = current_node.step_count + 1
                frontier.push(neighbor, neighbor_cost)

### Pseudocode cho A*:

function a_star(graph, start, goal):
    frontier = PriorityQueue()
    frontier.push(start, 0)
    explored = set()

    while not frontier.empty():
        current_node = frontier.pop()
        print(current_node.data)

        if current_node.data == goal:
            # Goal reached, return the path
            actions = get_actions_list(current_node)
            print("Actions:", actions)
            print("Step:", current_node.step_count)
            return

        explored.add(current_node.data)

        for neighbor in current_node.generate_child():
            if neighbor not in explored:
                neighbor_cost = current_node.step_count + 1
                heuristic_value = h(neighbor.data, goal)
                total_cost = neighbor_cost + heuristic_value
                frontier.push(neighbor, total_cost)




